In [69]:
import nltk
import numpy as np
import numpy.matlib
from nltk.grammar import *
from pprint import pprint

# Loading the Corpus and Grammars

In [2]:
# Loads the ATIS CNF grammar and the sentences. If the sentences file is not available,
# automatically downloads it and loads it

try:
    grammar = nltk.data.load('atis/atis-grammar-cnf.cfg')
    s = nltk.data.load('grammars/large_grammars/atis_sentences.txt')
    t = nltk.parse.util.extract_test_sentences(s)
    
except LookupError:
    nltk.download('large_grammars')
    
    grammar = nltk.data.load('atis/atis-grammar-cnf.cfg')
    s = nltk.data.load('grammars/large_grammars/atis_sentences.txt')
    t = nltk.parse.util.extract_test_sentences(s)

In [ ]:
with open('atis/atis-grammar-cnf.cfg','r') as string_cfg:
    grammar1 = nltk.grammar.CFG.fromstring(string_cfg.read())
grammar1

## Test Grammar

In [42]:
test_grammar_string = '''
S -> NP VP
NP -> Det N
NP -> NP PP
PP -> P NP
VP -> V NP
VP -> VP PP

NP -> 'I'
N -> 'elephant'
N -> 'pajamas'
V -> 'shot'
P -> 'in'
Det -> 'an'
Det -> 'my'
TESTE -> 'shot'
'''

test_grammar = nltk.grammar.CFG.fromstring(test_grammar_string)
test_grammar.productions()

[S -> NP VP,
 NP -> Det N,
 NP -> NP PP,
 PP -> P NP,
 VP -> V NP,
 VP -> VP PP,
 NP -> 'I',
 N -> 'elephant',
 N -> 'pajamas',
 V -> 'shot',
 P -> 'in',
 Det -> 'an',
 Det -> 'my',
 TESTE -> 'shot']

In [33]:
test_grammar.productions(rhs='shot')

[V -> 'shot']

In [7]:
# Return the grammar productions, filtered by the left-hand side
# or the first item in the right-hand side.
for prod in grammar1.productions(lhs=Nonterminal('order')):
    print(prod)
rule = grammar1.productions(lhs=Nonterminal('order'))



NameError: name 'grammar1' is not defined

In [20]:
print(test_grammar.productions(rhs=Nonterminal('elephant')))

[]


In [101]:
def CKY_recognizer(sentence, grammar):
    sent_list = sentence.split()
    n = len(sent_list)
    #Initializes a Matrix
    matrix = [[0 for i in range(n)] for j in range(n)]

    for i in range(1, n+1):
        # Adds all possible terminal nodes according to the grammar
        # need to add the fuction to retrieve in the grammar
        #print(matrix)
        matrix[n-i][i-1] = [rule.lhs()
                            for rule in grammar.productions(rhs=sent_list[i-1])]
    r=0
    for b in range(2, n+1):
        #print('b =', b)
        for i in range(0, n-b+1):
            #print('i =', i)
            
            # elemento a ser acessado na matrix [n-b-i][i]
            row = n-b-i
            column = i
            
            A = matrix[row][column]
            for k in range(1, b): # b = 3, k {1,2}
                # B fixa o valor da coluna e varia na linha
                B = matrix[row + (b-k)][column]
                C = matrix[row][column + k]
                A = B + C
                
                
                
                matrix[n-b-i][i] = B + C
                
                print('B =', B)
                print('C =', C)
                if input(''):
                    pass
#             matrix[n-b-i][i] = r
#             r+=1
            pprint(matrix)
            print('\n')
            
#             for k in range(1, b-1):
#                 B = matrix[i,i+k]
#                 C = matrix[i+k, i+b]
#                 # for each production rule A add A to the element matrix[i, i+b]
#                 matrix[i,i+b]
#                 print(matrix)
#             # Check the elments of the matrix 
                   
#             matrix[i][j] = "inside matrix"
#             print(matrix)
                
    return matrix

In [102]:
chart = CKY_recognizer('I shot an elephant in my pajamas', test_grammar)
pprint(chart)

B = [NP]
C = [V, TESTE]

[[0, 0, 0, 0, 0, 0, [N]],
 [0, 0, 0, 0, 0, [Det], 0],
 [0, 0, 0, 0, [P], 0, 0],
 [0, 0, 0, [N], 0, 0, 0],
 [0, 0, [Det], 0, 0, 0, 0],
 [[NP, V, TESTE], [V, TESTE], 0, 0, 0, 0, 0],
 [[NP], 0, 0, 0, 0, 0, 0]]


B = [V, TESTE]
C = [Det]

[[0, 0, 0, 0, 0, 0, [N]],
 [0, 0, 0, 0, 0, [Det], 0],
 [0, 0, 0, 0, [P], 0, 0],
 [0, 0, 0, [N], 0, 0, 0],
 [0, [V, TESTE, Det], [Det], 0, 0, 0, 0],
 [[NP, V, TESTE], [V, TESTE], 0, 0, 0, 0, 0],
 [[NP], 0, 0, 0, 0, 0, 0]]


B = [Det]
C = [N]

[[0, 0, 0, 0, 0, 0, [N]],
 [0, 0, 0, 0, 0, [Det], 0],
 [0, 0, 0, 0, [P], 0, 0],
 [0, 0, [Det, N], [N], 0, 0, 0],
 [0, [V, TESTE, Det], [Det], 0, 0, 0, 0],
 [[NP, V, TESTE], [V, TESTE], 0, 0, 0, 0, 0],
 [[NP], 0, 0, 0, 0, 0, 0]]


B = [N]
C = [P]

[[0, 0, 0, 0, 0, 0, [N]],
 [0, 0, 0, 0, 0, [Det], 0],
 [0, 0, 0, [N, P], [P], 0, 0],
 [0, 0, [Det, N], [N], 0, 0, 0],
 [0, [V, TESTE, Det], [Det], 0, 0, 0, 0],
 [[NP, V, TESTE], [V, TESTE], 0, 0, 0, 0, 0],
 [[NP], 0, 0, 0, 0, 0, 0]]


B = [P]
C = [Det

KeyboardInterrupt: Interrupted by user

In [94]:
pprint(chart)

[[[NP, V, TESTE, Det, N, P, Det, N],
  [V, TESTE, Det, N, P, Det, N],
  [Det, N, P, Det, N],
  [N, P, Det, N],
  [P, Det, N],
  [Det, N],
  [N]],
 [[NP, V, TESTE, Det, N, P, Det],
  [V, TESTE, Det, N, P, Det],
  [Det, N, P, Det],
  [N, P, Det],
  [P, Det],
  [Det],
  0],
 [[NP, V, TESTE, Det, N, P],
  [V, TESTE, Det, N, P],
  [Det, N, P],
  [N, P],
  [P],
  0,
  0],
 [[NP, V, TESTE, Det, N], [V, TESTE, Det, N], [Det, N], [N], 0, 0, 0],
 [[NP, V, TESTE, Det], [V, TESTE, Det], [Det], 0, 0, 0, 0],
 [[NP, V, TESTE], [V, TESTE], 0, 0, 0, 0, 0],
 [[NP], 0, 0, 0, 0, 0, 0]]


In [63]:
chart[0][1]

19

In [89]:
for i in range(1,2):
    print(i)

1
